In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Set the log level to keep the warnings down

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [3]:
# Load the training dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../input/cat-and-dog/training_set/training_set/',
    labels = 'inferred',
    batch_size = 64,
    image_size = (224,224),
    shuffle = True,
    seed = 82
)

In [14]:
# Load the test dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../input/cat-and-dog/test_set/test_set/',
    labels = 'inferred',
    batch_size = 64,
    image_size = (224,224),
    shuffle = True,
    seed = 82,
)

In [4]:
# Class name list generated by image_dataset_from_directory()
train_ds.class_names

In [5]:
# Plot out a few of the images and their labels so we can see what we're working with.
plt.figure(figsize=(10, 10))

for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train_ds.class_names[labels[i]])
        plt.axis("off")


In [6]:
# Input layer
inputs = keras.Input(shape=[224,224,3])

# Add some Conv2D and BatchNorm layers
x = layers.Conv2D(128, 3)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPool2D()(x)
x = layers.Dense(64, activation='relu')(x)

x = layers.Flatten()(x)

# Make a binary output layer with sigmoid activation
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create the model
model = keras.Model(inputs=inputs, outputs=outputs)

In [7]:
model.summary()

In [9]:
# Use BinaryCrossEntropy as a loss function since this is a binary classification problem
loss = keras.losses.BinaryCrossentropy()

# Use Adam optimizer with default learning rate
optimizer = keras.optimizers.Adam()

# The metric to monitor during training.
metrics = ['accuracy']

In [10]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.fit(train_ds, epochs=10, batch_size=64, verbose=1)

In [15]:
model.evaluate(test_ds, batch_size=64, verbose=2)

The overall accuracy of 65% isn't bad for a single model with very few images, epochs .. augments .. and no cross validation. Ok, maybe it's not quite a gold medal winning solution.
We can make it better!

One method to overcome the relatively low number of images is to use transfer learning to 'transfer knowledge' to our model.

These models have been trained on millions of images and can be easily loaded using the keras.applications API.

EfficientNet
ResNet
VGG
and many more ..
Let's load EfficientNetB0 and use its weights as our base model, although any of the keras.applications can be used.
Once loaded, we'll freeze the base model's weights, so they don't get updated during training.

In [16]:
# Load the base model
base_model = tf.keras.applications.EfficientNetB0()

# Freeze the existing layers
base_model.trainable = False

In [18]:
# Copy the input layer from the pretrained model to use as our input layer
inputs = base_model.layers[0].input

# Get all base model layers except the last two
outputs = base_model.layers[-2].output

# Add our final classification (output) layer
outputs = layers.Dense(1, activation='sigmoid')(outputs)

# Create the model
model2 = keras.Model(inputs=inputs, outputs=outputs)

In [19]:
# Use the same loss, opt and metrics
loss = keras.losses.BinaryCrossentropy()
optimizer = keras.optimizers.Adam()
metrics = ['accuracy']

In [20]:
model2.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model2.fit(train_ds, epochs=10, batch_size=64, verbose=1)

In [21]:
model2.evaluate(test_ds, batch_size=64, verbose=1)

When faced with an image classification problem where training images are few, Tensorflow makes it easy to incorporate transfer learning into your solution.
By simplifying loading of pre-trained weights and interfacing with various architectures, we can quickly create accurate image classification models.